In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# file paths:
school_data_path = Path("Resources/schools_complete.csv")
student_data_path = Path("Resources/students_complete.csv")

# read in the csv files:
school_data = pd.read_csv(school_data_path)
student_data = pd.read_csv(student_data_path)

# join the data into a single dataset:
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

# take a look at the dataset:
school_data_complete

,Student ID,student_name,gender,year,school_name,reading_score,maths_score,School ID,type,size,budget
0,0,Paul Bradley,M,9,Huang High School,96,94,0,Government,2917,1910635
1,1,Victor Smith,M,12,Huang High School,90,43,0,Government,2917,1910635
2,2,Kevin Rodriguez,M,12,Huang High School,41,76,0,Government,2917,1910635
3,3,Richard Scott,M,12,Huang High School,89,86,0,Government,2917,1910635
4,4,Bonnie Ray,F,9,Huang High School,87,69,0,Government,2917,1910635
...,...,...,...,...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12,Thomas High School,51,48,14,Independent,1635,1043130
39166,39166,Dawn Bell,F,10,Thomas High School,81,89,14,Independent,1635,1043130
39167,39167,Rebecca Tanner,F,9,Thomas High School,99,99,14,Independent,1635,1043130
39168,39168,Desiree Kidd,F,10,Thomas High School,72,77,14,Independent,1635,1043130


## Local Government Area Summary

In [3]:
# - Calculate the total number of schools
total_schools = school_data_complete['school_name'].nunique()

# - Calculate the total number of students
total_students = school_data_complete['Student ID'].nunique()

# - Calculate the total budget
total_budget = school_data_complete['budget'].unique().sum()

# - Calculate the average maths score and average reading score
average_maths_score = school_data_complete['maths_score'].mean()
average_reading_score = school_data_complete['reading_score'].mean()

# - Calculate the percentage of students with a passing maths score (50 or greater)
passing_maths_percentage = (school_data_complete['maths_score'] >= 50).mean() * 100

# - Calculate the percentage of students with a passing reading score (50 or greater)
passing_reading_percentage = (school_data_complete['reading_score'] >= 50).mean() * 100

# - Calculate the percentage of students who passed maths and reading (% Overall Passing)
overall_passing_percentage = ((school_data_complete['maths_score'] >= 50) & (school_data_complete['reading_score'] >= 50)).mean() * 100

# - Create a DataFrame to hold the results
results_df = pd.DataFrame({
    'Total Schools': [total_schools],
    'Total Students': [f"{total_students:,}"],
    'Total Budget': [f"${total_budget:,.2f}"],
    'Average Maths Score': [average_maths_score],
    'Average Reading Score': [average_reading_score],
    '% Passing Maths': [passing_maths_percentage],
    '% Passing Reading': [passing_reading_percentage],
    '% Overall Passing': [overall_passing_percentage]
})

# Display the results DataFrame
results_df

,Total Schools,Total Students,Total Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",70.338192,69.980138,86.078632,84.426857,72.808272


From the Local Government Area Summary above we observe that:
- There are a total of 15 schools, all of which have 39,170 students combined
- 72% of students passed both maths and reading

## School Summary

In [4]:
# group the data by school name to calculate metrics
grouped_schools = school_data_complete.groupby('school_name')

# - Calculate the total students per school
total_students = grouped_schools['Student ID'].count()

# - Calculate the school type
school_type = grouped_schools['type'].first()

# - Calculate the total school budget per school
total_school_budget = grouped_schools['budget'].first()

# - Calculate the per student budget per school
per_student_budget = total_school_budget / total_students

# - Calculate the average math and reading scores per school
average_math_score = grouped_schools['maths_score'].mean()
average_reading_score = grouped_schools['reading_score'].mean()

# - Calculate the percentage passing math, reading, and overall passing
passing_math = school_data_complete[school_data_complete['maths_score'] >= 50].groupby('school_name')['Student ID'].count() / total_students * 100
passing_reading = school_data_complete[school_data_complete['reading_score'] >= 50].groupby('school_name')['Student ID'].count() / total_students * 100
overall_passing = school_data_complete[(school_data_complete['maths_score'] >= 50) & (school_data_complete['reading_score'] >= 50)].groupby('school_name')['Student ID'].count() / total_students * 100

# - Create the overview dataframe
per_school_summary = pd.DataFrame({
    'School Type': school_type,
    'Total Students': total_students,
    'Total School Budget': total_school_budget,
    'Per Student Budget': per_student_budget,
    'Average Maths Score': average_math_score,
    'Average Reading Score': average_reading_score,
    '% Passing Maths': passing_math,
    '% Passing Reading': passing_reading,
    '% Overall Passing': overall_passing
})

# formatting the columns
per_school_summary['Total School Budget'] = per_school_summary['Total School Budget'].map('${:,.2f}'.format)
per_school_summary['Per Student Budget'] = per_school_summary['Per Student Budget'].map('${:.2f}'.format)
# remove index column name
per_school_summary.index.name = ""

# Display the result
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
,,,,,,,,,
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Figueroa High School,Government,2949,"$1,884,411.00",$639.00,68.698542,69.077993,81.654798,82.807731,67.650051
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Holden High School,Independent,427,"$248,087.00",$581.00,72.583138,71.660422,89.929742,88.524590,78.922717
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766


From the School Summary above we note that:
- There are 8 independent schools and 7 government schools
- Holden High School has the least number of students (427)

## Top Performing Schools (By % Overall Passing)

In [5]:
# - Sort and display the top five performing schools by % overall passing
top_schools = per_school_summary.sort_values(by='% Overall Passing', ascending=False).head(5)

# Display the result
top_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
,,,,,,,,,
Griffin High School,Independent,1468,"$917,500.00",$625.00,71.788147,71.245232,91.212534,88.487738,81.335150
Cabrera High School,Independent,1858,"$1,081,356.00",$582.00,71.657158,71.359526,90.850377,89.074273,80.785791
Bailey High School,Government,4976,"$3,124,928.00",$628.00,72.352894,71.008842,91.639871,87.379421,80.084405
Wright High School,Independent,1800,"$1,049,400.00",$583.00,72.047222,70.969444,91.777778,86.666667,79.722222
Rodriguez High School,Government,3999,"$2,547,363.00",$637.00,72.047762,70.935984,90.797699,87.396849,79.419855


From the above table we conclude that:
- The top school Griffin High School with an overall passing percentage of 81.34%
- The deviations of overall passing percentages of the top 5 schools are very small.

## Bottom Performing Schools (By % Overall Passing)

In [6]:
# - Sort and display the five worst-performing schools by % overall passing
bottom_schools = per_school_summary.sort_values(by='% Overall Passing', ascending=True).head(5)

# Display the result
bottom_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
,,,,,,,,,
Hernandez High School,Government,4635,"$3,022,020.00",$652.00,68.874865,69.186408,80.949299,81.877023,66.364617
Huang High School,Government,2917,"$1,910,635.00",$655.00,68.935207,68.910525,81.693521,81.453548,66.712376
Johnson High School,Government,4761,"$3,094,650.00",$650.00,68.843100,69.039277,82.062592,81.978576,67.191766
Wilson High School,Independent,2283,"$1,319,574.00",$578.00,69.170828,68.876916,82.785808,81.296540,67.455103
Ford High School,Government,2739,"$1,763,916.00",$644.00,69.091274,69.572472,82.438846,82.219788,67.469880


Here are my conclusions from the bottom performing schools summary:
- Most of the bottom performing schools are government schools
- In terms of the overall passing percentage, there appears to be a marginal difference between the bottom performing schools and the top performing ones.

## Maths Scores by Year

In [7]:
# Create separate Series for each year level
year_9_scores = school_data_complete[school_data_complete['year'] == 9].groupby('school_name')['maths_score'].mean()
year_10_scores = school_data_complete[school_data_complete['year'] == 10].groupby('school_name')['maths_score'].mean()
year_11_scores = school_data_complete[school_data_complete['year'] == 11].groupby('school_name')['maths_score'].mean()
year_12_scores = school_data_complete[school_data_complete['year'] == 12].groupby('school_name')['maths_score'].mean()

# Combine the Series into a DataFrame
math_scores_by_year = pd.DataFrame({
    'Year 9': year_9_scores,
    'Year 10': year_10_scores,
    'Year 11': year_11_scores,
    'Year 12': year_12_scores
})

# remove index column name
math_scores_by_year.index.name = ""

# Display the result
math_scores_by_year

,Year 9,Year 10,Year 11,Year 12
,,,,
Bailey High School,72.493827,71.897498,72.374900,72.675097
Cabrera High School,72.321970,72.437768,71.008299,70.604712
Figueroa High School,68.477804,68.331586,68.811001,69.325282
Ford High School,69.021609,69.387006,69.248862,68.617811
Griffin High School,72.789731,71.093596,71.692521,71.469178
Hernandez High School,68.586831,68.867156,69.154412,68.985075
Holden High School,70.543307,75.105263,71.640777,73.409639
Huang High School,69.081754,68.533246,69.431345,68.639316
Johnson High School,69.469286,67.990220,68.637730,69.287393


## Reading Score by Year

In [8]:
# Create separate Series for each year level for reading scores
year_9_reading_scores = school_data_complete[school_data_complete['year'] == 9].groupby('school_name')['reading_score'].mean()
year_10_reading_scores = school_data_complete[school_data_complete['year'] == 10].groupby('school_name')['reading_score'].mean()
year_11_reading_scores = school_data_complete[school_data_complete['year'] == 11].groupby('school_name')['reading_score'].mean()
year_12_reading_scores = school_data_complete[school_data_complete['year'] == 12].groupby('school_name')['reading_score'].mean()

# Combine the Series into a DataFrame for reading scores
reading_scores_by_year = pd.DataFrame({
    'Year 9': year_9_reading_scores,
    'Year 10': year_10_reading_scores,
    'Year 11': year_11_reading_scores,
    'Year 12': year_12_reading_scores
})

# remove index column name
reading_scores_by_year.index.name = ""

# Display the result for reading scores
reading_scores_by_year

,Year 9,Year 10,Year 11,Year 12
,,,,
Bailey High School,70.901920,70.848265,70.317346,72.195525
Cabrera High School,71.172348,71.328326,71.201245,71.856021
Figueroa High School,70.261682,67.677588,69.152327,69.082126
Ford High School,69.615846,68.988701,70.735964,68.849722
Griffin High School,72.026895,70.746305,72.385042,69.434932
Hernandez High School,68.477569,70.621842,68.418199,69.244136
Holden High School,71.598425,71.096491,73.310680,70.481928
Huang High School,68.670616,69.516297,68.740638,68.671795
Johnson High School,68.719286,69.295029,69.969115,67.992521


## Scores by School Spending

In [9]:
# - Create bins and labels for spending ranges
spending_bins = [0, 585, 630, 645, 680]
spending_labels = ["<$585", "$585-630", "$630-645", "$645-680"]

# Add a new column to the 'per_school_summary' DataFrame with the spending range for each school
per_school_summary['Spending Ranges (Per Student)'] = pd.cut(
    per_school_summary['Total School Budget'].str.replace('[$,]', '', regex=True).astype(float) / per_school_summary['Total Students'],
    spending_bins, labels=spending_labels)

# - Group the schools based on the spending ranges
grouped_spending = per_school_summary.groupby('Spending Ranges (Per Student)')

# - Calculate the average math and reading scores
average_math_score = grouped_spending['Average Maths Score'].mean()
average_reading_score = grouped_spending['Average Reading Score'].mean()

# - Calculate the percentage passing math, reading, and overall passing rate
passing_math = grouped_spending['% Passing Maths'].mean()
passing_reading = grouped_spending['% Passing Reading'].mean()
overall_passing_rate = grouped_spending['% Overall Passing'].mean()

# - Create a DataFrame to display the results
spending_summary_df = pd.DataFrame({
    'Average Maths Score': average_math_score,
    'Average Reading Score': average_reading_score,
    '% Passing Maths': passing_math,
    '% Passing Reading': passing_reading,
    '% Overall Passing': overall_passing_rate
})

# Round all columns to two decimal places
spending_summary_df = spending_summary_df.round(2)

# Display the result
spending_summary_df

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,71.36,70.72,88.84,86.39,76.72
$585-630,72.07,71.03,91.52,87.29,79.88
$630-645,69.85,69.84,84.69,83.76,71.00
$645-680,68.88,69.05,81.57,81.77,66.76


From the School Spending summary table above we can see that:
- Schools with spending ranges (per student) of $585-630 have the highest overall passing percentage
- Schools with the highest spending ranges (per student) also have the least overall passing percentage

## Scores by School Size

In [10]:
# - Create bins and labels for school size ranges
size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Add a new column to the 'per_school_summary' DataFrame with the school size range for each school
per_school_summary['School Size'] = pd.cut(per_school_summary['Total Students'], size_bins, labels=size_labels)

# - Group the schools based on the school size ranges
grouped_size = per_school_summary.groupby('School Size')

# - Calculate the average math and reading scores
average_math_score = grouped_size['Average Maths Score'].mean()
average_reading_score = grouped_size['Average Reading Score'].mean()

# - Calculate the percentage passing math, reading, and overall passing rate
passing_math = grouped_size['% Passing Maths'].mean()
passing_reading = grouped_size['% Passing Reading'].mean()
overall_passing_rate = grouped_size['% Overall Passing'].mean()

# - Create a DataFrame to display the results
size_summary = pd.DataFrame({
    'Average Maths Score': average_math_score,
    'Average Reading Score': average_reading_score,
    '% Passing Maths': passing_math,
    '% Passing Reading': passing_reading,
    '% Overall Passing': overall_passing_rate
})

# Display the result
size_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),72.335748,71.636864,90.806867,87.557513,79.066348
Medium (1000-2000),71.421650,70.720164,89.846560,86.714149,78.039785
Large (2000-5000),69.751809,69.576052,84.252804,83.301185,70.293507


It is evident that:
- Schools with small sizes have the highest overall passing percentage
- Schools with large sizes have the least average scores in both maths and reading

## Scores by School Type

In [12]:
# - Group the data by school type
grouped_type = per_school_summary.groupby('School Type')

# - Calculate the average math and reading scores
average_math_score = grouped_type['Average Maths Score'].mean()
average_reading_score = grouped_type['Average Reading Score'].mean()

# - Calculate the percentage passing math, reading, and overall passing rate
passing_math = grouped_type['% Passing Maths'].mean()
passing_reading = grouped_type['% Passing Reading'].mean()
overall_passing_rate = grouped_type['% Overall Passing'].mean()

# - Create a DataFrame to display the results
type_summary = pd.DataFrame({
    'Average Maths Score': average_math_score,
    'Average Reading Score': average_reading_score,
    '% Passing Maths': passing_math,
    '% Passing Reading': passing_reading,
    '% Overall Passing': overall_passing_rate
})

# Display the result
type_summary

,Average Maths Score,Average Reading Score,% Passing Maths,% Passing Reading,% Overall Passing
School Type,,,,,
Government,69.834806,69.675929,84.462375,83.587562,70.698993
Independent,71.368822,70.718933,89.204043,86.247789,76.973340


From the above summary we can conclude that:
- Generally, government schools have lower average scores for both maths and reading
- Consequently, government schools have lower overall passing percentage compared to the independent schools

## Written Report

### Summary

Overall, this analysis provides valuable insights into school performance, enabling educators and policymakers to make informed decisions to improve educational outcomes for students.

Here are the main findings:

- **Total Schools and Students:** There are a total of 15 schools in the dataset, with a combined student population of 39,170.

- **Passing Rates:** Approximately 86.08% of students passed math, while 84.43% passed reading. An impressive 72.81% of students passed both math and reading, indicating a strong overall performance.

### Conclusions

1. **Consistency in Top Schools:** The top-performing schools exhibit a consistent level of excellence, with minimal variations in their overall passing percentages. This suggests that these schools have effective strategies in place to maintain high academic standards.

2. **Impact of School Type:** Independent schools outperform government schools in terms of average scores and overall passing rates. This highlights the potential benefits of different governance models and the need for targeted improvements in government-funded education.